In [ ]:
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', 200)

from google.colab import drive
drive.mount('/content/gdrive')

PROJECT_PATH  = '/content/gdrive/MyDrive/OperAI/final-project'
DATA_PATH     = os.path.join(PROJECT_PATH, 'data')

Mounted at /content/gdrive


In [ ]:
vax   = pd.read_csv(os.path.join(DATA_PATH, 'COVID-19_Vaccinations_in_the_United_States_County.csv'))

<ipython-input-2-9ccf2f2c7906>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  vax   = pd.read_csv(os.path.join(DATA_PATH, 'COVID-19_Vaccinations_in_the_United_States_County.csv'))


# Vax

In [ ]:
vax.columns = vax.columns.str.lower()

In [ ]:
vax['date'] = pd.to_datetime(vax['date'])

In [ ]:
vax.tail()

,date,fips,mmwr_week,recip_county,recip_state,completeness_pct,administered_dose1_recip,administered_dose1_pop_pct,administered_dose1_recip_5plus,administered_dose1_recip_5pluspop_pct,administered_dose1_recip_12plus,administered_dose1_recip_12pluspop_pct,administered_dose1_recip_18plus,administered_dose1_recip_18pluspop_pct,administered_dose1_recip_65plus,administered_dose1_recip_65pluspop_pct,series_complete_yes,series_complete_pop_pct,series_complete_5plus,series_complete_5pluspop_pct,series_complete_5to17,series_complete_5to17pop_pct,series_complete_12plus,series_complete_12pluspop_pct,series_complete_18plus,series_complete_18pluspop_pct,series_complete_65plus,series_complete_65pluspop_pct,booster_doses,booster_doses_vax_pct,booster_doses_5plus,booster_doses_5plus_vax_pct,booster_doses_12plus,booster_doses_12plus_vax_pct,booster_doses_18plus,booster_doses_18plus_vax_pct,booster_doses_50plus,booster_doses_50plus_vax_pct,booster_doses_65plus,booster_doses_65plus_vax_pct,second_booster_50plus,second_booster_50plus_vax_pct,second_booster_65plus,second_booster_65plus_vax_pct,svi_ctgy,series_complete_pop_pct_svi,series_complete_5pluspop_pct_svi,series_complete_5to17pop_pct_svi,series_complete_12pluspop_pct_svi,series_complete_18pluspop_pct_svi,series_complete_65pluspop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,series_complete_5pluspop_pct_ur_equity,series_complete_5to17pop_pct_ur_equity,series_complete_12pluspop_pct_ur_equity,series_complete_18pluspop_pct_ur_equity,series_complete_65pluspop_pct_ur_equity,booster_doses_vax_pct_svi,booster_doses_12plusvax_pct_svi,booster_doses_18plusvax_pct_svi,booster_doses_65plusvax_pct_svi,booster_doses_vax_pct_ur_equity,booster_doses_12plusvax_pct_ur_equity,booster_doses_18plusvax_pct_ur_equity,booster_doses_65plusvax_pct_ur_equity,census2019,census2019_5pluspop,census2019_5to17pop,census2019_12pluspop,census2019_18pluspop,census2019_65pluspop,bivalent_booster_5plus,bivalent_booster_5plus_pop_pct,bivalent_booster_12plus,bivalent_booster_12plus_pop_pct,bivalent_booster_18plus,bivalent_booster_18plus_pop_pct,bivalent_booster_65plus,bivalent_booster_65plus_pop_pct
1962776,2020-12-13,39001,51,Adams County,OH,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27698.0,NaN,NaN,23476.0,21111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962777,2020-12-13,18061,51,Harrison County,IN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40515.0,NaN,NaN,34713.0,31495.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962778,2020-12-13,48321,51,Matagorda County,TX,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36643.0,NaN,NaN,30315.0,27278.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962779,2020-12-13,54073,51,Pleasants County,WV,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7460.0,NaN,NaN,6570.0,6077.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962780,2020-12-13,26061,51,Houghton County,MI,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35684.0,NaN,NaN,31033.0,28440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
vax.loc[vax['fips'].str.contains('[^0-9]').isna(), 'fips']

1957888    29047
1957889    42067
1957890    18013
1957891    39171
1957892     5017
           ...  
1962776    39001
1962777    18061
1962778    48321
1962779    54073
1962780    26061
Name: fips, Length: 4893, dtype: object

There are 34,829 observations (1.8% of the data where the county is not known). I think we have no choice but to drop these, but we can potentially look into which states are most common.

In [ ]:
total_unknown_county_obs = (vax['fips']=='UNK').sum()
print(f"{total_unknown_county_obs} ({total_unknown_county_obs / vax.shape[0]:.01%})")

34829 (1.8%)


In [ ]:
vax = vax.loc[vax['fips']!='UNK', :]
vax.shape

(1927952, 80)

In [ ]:
vax['fips'] = vax['fips'].astype(int).apply(lambda code: f'{code:05d}')

In [ ]:
vax['fips'].tail()

1962776    39001
1962777    18061
1962778    48321
1962779    54073
1962780    26061
Name: fips, dtype: object

In [ ]:
vax.isna().sum()

date                                     0
fips                                     0
mmwr_week                                0
recip_county                             0
recip_state                              0
                                    ...   
bivalent_booster_12plus_pop_pct    1844622
bivalent_booster_18plus            1844544
bivalent_booster_18plus_pop_pct    1844622
bivalent_booster_65plus            1844544
bivalent_booster_65plus_pop_pct    1844622
Length: 80, dtype: int64

# Death

In [ ]:
death = pd.read_csv(os.path.join(DATA_PATH, 'time_series_covid19_deaths_US.csv'))

## Drop obs with null FIPS

In [ ]:
death.columns = death.columns.str.lower()

First, check whether there are any FIPS codes that are N/A. Only Kansas City has non-zero population, but this population should be included in Jackson County, MO, which is in the dataset (`FIPS == 29095`, see `death.loc[death['fips'] == 29095]`). Therefore, we should feel comfortable dropping all obs with N/A FIPS codes.

In [ ]:
print(death.loc[death['fips'].isna(), ['combined_key', 'population']].value_counts())
print()
print(death.loc[death['fips'] == 29095, 'combined_key'])

combined_key                                             population
Bear River, Utah, US                                     0             1
Central Utah, Utah, US                                   0             1
Dukes and Nantucket,Massachusetts,US                     0             1
Federal Correctional Institution (FCI), Michigan, US     0             1
Kansas City,Missouri,US                                  488943        1
Michigan Department of Corrections (MDOC), Michigan, US  0             1
Southeast Utah, Utah, US                                 0             1
Southwest Utah, Utah, US                                 0             1
TriCounty, Utah, US                                      0             1
Weber-Morgan, Utah, US                                   0             1
dtype: int64

1589    Jackson, Missouri, US
Name: combined_key, dtype: object


In [ ]:
death = death.dropna(subset='fips')

Now we can safely convert to string FIPS codes.

In [ ]:
death['fips'] = death['fips'].astype(int).apply(lambda code: f'{code:05d}')

death['fips'].head()

0    01001
1    01003
2    01005
3    01007
4    01009
Name: fips, dtype: object

In [ ]:
print(list(death.columns))
death_id_cols = ['uid', 'iso2', 'iso3', 'code3', 'fips', 'admin2',
                 'province_state', 'country_region', 'lat',
                 'long_', 'combined_key', 'population']

['uid', 'iso2', 'iso3', 'code3', 'fips', 'admin2', 'province_state', 'country_region', 'lat', 'long_', 'combined_key', 'population', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19', '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23',

In [ ]:
death_long = death.melt(
    id_vars=death_id_cols,
    value_name='deaths',
    var_name='date',
    ignore_index=True
)

death_long.head()

,uid,iso2,iso3,code3,fips,admin2,province_state,country_region,lat,long_,combined_key,population,date,deaths
0,84001001,US,USA,840,01001,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869,2020-01-22,0
1,84001003,US,USA,840,01003,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",223234,2020-01-22,0
2,84001005,US,USA,840,01005,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",24686,2020-01-22,0
3,84001007,US,USA,840,01007,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",22394,2020-01-22,0
4,84001009,US,USA,840,01009,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",57826,2020-01-22,0


In [ ]:
death_long['date'] = pd.to_datetime(death_long['date'])

In [ ]:
death_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808476 entries, 0 to 3808475
Data columns (total 14 columns):
 #   Column          Dtype         
---  ------          -----         
 0   uid             int64         
 1   iso2            object        
 2   iso3            object        
 3   code3           int64         
 4   fips            object        
 5   admin2          object        
 6   province_state  object        
 7   country_region  object        
 8   lat             float64       
 9   long_           float64       
 10  combined_key    object        
 11  population      int64         
 12  date            datetime64[ns]
 13  deaths          int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(7)
memory usage: 406.8+ MB


In [ ]:
death_long_subset = death_long.loc[:, ['fips', 'combined_key', 'date', 'population', 'deaths']]
death_long_subset.head()

,fips,combined_key,date,population,deaths
0,01001,"Autauga, Alabama, US",2020-01-22,55869,0
1,01003,"Baldwin, Alabama, US",2020-01-22,223234,0
2,01005,"Barbour, Alabama, US",2020-01-22,24686,0
3,01007,"Bibb, Alabama, US",2020-01-22,22394,0
4,01009,"Blount, Alabama, US",2020-01-22,57826,0


# Merging vax and deaths

In [ ]:
df = vax.join(
    death_long_subset.set_index(['fips', 'date']),
    on=['fips', 'date'],
    how='left',
    lsuffix='_vax',
    rsuffix='_death',
    sort=True,
    validate='m:1'
)
df.head()

,date,fips,mmwr_week,recip_county,recip_state,completeness_pct,administered_dose1_recip,administered_dose1_pop_pct,administered_dose1_recip_5plus,administered_dose1_recip_5pluspop_pct,administered_dose1_recip_12plus,administered_dose1_recip_12pluspop_pct,administered_dose1_recip_18plus,administered_dose1_recip_18pluspop_pct,administered_dose1_recip_65plus,administered_dose1_recip_65pluspop_pct,series_complete_yes,series_complete_pop_pct,series_complete_5plus,series_complete_5pluspop_pct,series_complete_5to17,series_complete_5to17pop_pct,series_complete_12plus,series_complete_12pluspop_pct,series_complete_18plus,series_complete_18pluspop_pct,series_complete_65plus,series_complete_65pluspop_pct,booster_doses,booster_doses_vax_pct,booster_doses_5plus,booster_doses_5plus_vax_pct,booster_doses_12plus,booster_doses_12plus_vax_pct,booster_doses_18plus,booster_doses_18plus_vax_pct,booster_doses_50plus,booster_doses_50plus_vax_pct,booster_doses_65plus,booster_doses_65plus_vax_pct,second_booster_50plus,second_booster_50plus_vax_pct,second_booster_65plus,second_booster_65plus_vax_pct,svi_ctgy,series_complete_pop_pct_svi,series_complete_5pluspop_pct_svi,series_complete_5to17pop_pct_svi,series_complete_12pluspop_pct_svi,series_complete_18pluspop_pct_svi,series_complete_65pluspop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,series_complete_5pluspop_pct_ur_equity,series_complete_5to17pop_pct_ur_equity,series_complete_12pluspop_pct_ur_equity,series_complete_18pluspop_pct_ur_equity,series_complete_65pluspop_pct_ur_equity,booster_doses_vax_pct_svi,booster_doses_12plusvax_pct_svi,booster_doses_18plusvax_pct_svi,booster_doses_65plusvax_pct_svi,booster_doses_vax_pct_ur_equity,booster_doses_12plusvax_pct_ur_equity,booster_doses_18plusvax_pct_ur_equity,booster_doses_65plusvax_pct_ur_equity,census2019,census2019_5pluspop,census2019_5to17pop,census2019_12pluspop,census2019_18pluspop,census2019_65pluspop,bivalent_booster_5plus,bivalent_booster_5plus_pop_pct,bivalent_booster_12plus,bivalent_booster_12plus_pop_pct,bivalent_booster_18plus,bivalent_booster_18plus_pop_pct,bivalent_booster_65plus,bivalent_booster_65plus_pop_pct,combined_key,population,deaths
1961617,2020-12-13,01001,51,Autauga County,AL,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55869.0,NaN,NaN,47574.0,42904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga, Alabama, US",55869.0,41.0
1957919,2020-12-14,01001,51,Autauga County,AL,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55869.0,NaN,NaN,47574.0,42904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga, Alabama, US",55869.0,41.0
1955846,2020-12-15,01001,51,Autauga County,AL,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55869.0,NaN,NaN,47574.0,42904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga, Alabama, US",55869.0,43.0
1950822,2020-12-16,01001,51,Autauga County,AL,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55869.0,NaN,NaN,47574.0,42904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autauga, Alabama, US",55869.0,43.0
1947776,2020-12-17,01001,51,Autauga County,AL,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN

In [ ]:
df.shape

(1927952, 83)

## Note on unmatched deaths data
As we can see in the cells below, there are 31,372 unmatched death values. However, 92% of these unmatched ones are simply due to the fact that death data does not extend out as far as the vaccination data (see below).

As such, for now, we will drop unmatched death data obs.

In [ ]:
df.isna().sum()

date                                     0
fips                                     0
mmwr_week                                0
recip_county                             0
recip_state                              0
                                    ...   
bivalent_booster_65plus            1844544
bivalent_booster_65plus_pop_pct    1844622
combined_key                         31372
population                           31372
deaths                               31372
Length: 83, dtype: int64

In [ ]:
df[df['deaths'].isna()]

,date,fips,mmwr_week,recip_county,recip_state,completeness_pct,administered_dose1_recip,administered_dose1_pop_pct,administered_dose1_recip_5plus,administered_dose1_recip_5pluspop_pct,administered_dose1_recip_12plus,administered_dose1_recip_12pluspop_pct,administered_dose1_recip_18plus,administered_dose1_recip_18pluspop_pct,administered_dose1_recip_65plus,administered_dose1_recip_65pluspop_pct,series_complete_yes,series_complete_pop_pct,series_complete_5plus,series_complete_5pluspop_pct,series_complete_5to17,series_complete_5to17pop_pct,series_complete_12plus,series_complete_12pluspop_pct,series_complete_18plus,series_complete_18pluspop_pct,series_complete_65plus,series_complete_65pluspop_pct,booster_doses,booster_doses_vax_pct,booster_doses_5plus,booster_doses_5plus_vax_pct,booster_doses_12plus,booster_doses_12plus_vax_pct,booster_doses_18plus,booster_doses_18plus_vax_pct,booster_doses_50plus,booster_doses_50plus_vax_pct,booster_doses_65plus,booster_doses_65plus_vax_pct,second_booster_50plus,second_booster_50plus_vax_pct,second_booster_65plus,second_booster_65plus_vax_pct,svi_ctgy,series_complete_pop_pct_svi,series_complete_5pluspop_pct_svi,series_complete_5to17pop_pct_svi,series_complete_12pluspop_pct_svi,series_complete_18pluspop_pct_svi,series_complete_65pluspop_pct_svi,metro_status,series_complete_pop_pct_ur_equity,series_complete_5pluspop_pct_ur_equity,series_complete_5to17pop_pct_ur_equity,series_complete_12pluspop_pct_ur_equity,series_complete_18pluspop_pct_ur_equity,series_complete_65pluspop_pct_ur_equity,booster_doses_vax_pct_svi,booster_doses_12plusvax_pct_svi,booster_doses_18plusvax_pct_svi,booster_doses_65plusvax_pct_svi,booster_doses_vax_pct_ur_equity,booster_doses_12plusvax_pct_ur_equity,booster_doses_18plusvax_pct_ur_equity,booster_doses_65plusvax_pct_ur_equity,census2019,census2019_5pluspop,census2019_5to17pop,census2019_12pluspop,census2019_18pluspop,census2019_65pluspop,bivalent_booster_5plus,bivalent_booster_5plus_pop_pct,bivalent_booster_12plus,bivalent_booster_12plus_pop_pct,bivalent_booster_18plus,bivalent_booster_18plus_pop_pct,bivalent_booster_65plus,bivalent_booster_65plus_pop_pct,combined_key,population,deaths
26942,2023-03-15,01001,11,Autauga County,AL,92.6,32381.0,58.0,32300.0,61.4,31343.0,65.9,29179.0,68.0,8079.0,90.5,25720.0,46.0,25698.0,48.9,2442.0,25.2,24998.0,52.5,23256.0,54.2,6629.0,74.3,9756.0,37.9,9752.0,37.9,9699.0,38.8,9385.0,40.4,6918.0,51.2,4023.0,60.7,2589.0,37.4,1691.0,42.0,C,9.0,9.0,9.0,10.0,10.0,11.0,Metro,1.0,1.0,1.0,2.0,2.0,3.0,10.0,10.0,11.0,10.0,2.0,2.0,3.0,2.0,55869.0,52592.0,9688.0,47574.0,42904.0,8924.0,3450.0,6.6,3437.0,7.2,3349.0,7.8,1711.0,19.2,NaN,NaN,NaN
23400,2023-03-22,01001,12,Autauga County,AL,92.6,32381.0,58.0,32300.0,61.4,31343.0,65.9,29179.0,68.0,8079.0,90.5,25728.0,46.1,25702.0,48.9,2442.0,25.2,25002.0,52.6,23260.0,54.2,6629.0,74.3,9763.0,37.9,9759.0,38.0,9705.0,38.8,9388.0,40.4,6919.0,51.2,4024.0,60.7,2591.0,37.4,1692.0,42.0,C,9.0,9.0,9.0,10.0,10.0,11.0,Metro,1.0,1.0,1.0,2.0,2.0,3.0,10.0,10.0,11.0,10.0,2.0,2.0,3.0,2.0,55869.0,52592.0,9688.0,47574.0,42904.0,8924.0,3464.0,6.6,3450.0,7.3,3359.0,7.8,1716.0,19.2,NaN,NaN,NaN
21480,2023-03-29,01001,13,Autauga County,AL,92.6,32388.0,58.0,32307.0,61.4,31349.0,65.9,29184.0,68.0,8080.0,90.5,25731.0,46.1,25705.0,48.9,2442.0,25.2,25005.0,52.6,23263.0,54.2,6630.0,74.3,9767.0,38.0,9763.0,38.0,9709.0,38.8,9392.0,40.4,6922.0,51.2,4026.0,60.7,2605.0,37.6,1700.0,42.2,C,9.0,9.0,9.0,10.0,10.0,11.0,Metro,1.0,1.0,1.0,2.0,2.0,3.0,10.0,10.0,11.0,10.0,2.0,2.0,3.0,2.0,55869.0,52592.0,9688.0,47574.0,42904.0,8924.0,3495.0,6.6,3481.0,7.3,3390.0,7.9,1731.0,19.4,NaN,NaN,NaN
18383,2023-04-05,01001,14,Autauga County,AL,92.6,32397.0,58.0,32316.0,61.4,31357.0,65.9,29191.0,68.0,8080.0,90.5,25734.0,46.1,25708.0,48.9,2443.0,25.2,25007.0,52.6,23265.0,54.2,6630.0,74.3,9771.0,38.0,9767.0,38.0,9713.0,38.8,9396.0,40.4,6925.0,51.2,4029.0,60.8,2605.0,37.6,1700.0,42.2,C,9.0,9.0,9.0,10.0,10.0,11.0,Metro,1.0,1.0,1.0,2.0,2.0,3.0,10.0,10.0,11.0,10.0,2.0,2.0,3.0,

In [ ]:
deaths_max_date = death_long_subset['date'].max()
deaths_max_date

Timestamp('2023-03-09 00:00:00')

In [ ]:
df.loc[df['deaths'].isna(), 'date'].describe(datetime_is_numeric=True)

count                            31372
mean     2023-03-01 19:22:12.500318720
min                2020-12-13 00:00:00
25%                2023-03-22 00:00:00
50%                2023-04-12 00:00:00
75%                2023-04-26 00:00:00
max                2023-05-10 00:00:00
Name: date, dtype: object

In [ ]:
deaths_null_mask = df['deaths'].isna()
date_after_last_mask = df['date'] > deaths_max_date
both = (deaths_null_mask & date_after_last_mask)

print(f"All null deaths: {deaths_null_mask.sum()}")
print(f"Obs after last date in death data: {date_after_last_mask.sum()}")
print(f"Both: {both.sum()} ({both.sum()/deaths_null_mask.sum():.02%})")

All null deaths: 31372
Obs after last date in death data: 29016
Both: 29016 (92.49%)


In [ ]:
df = df.loc[df['deaths'].notna(), :]

In [ ]:
df.isna().sum()

date                                     0
fips                                     0
mmwr_week                                0
recip_county                             0
recip_state                              0
                                    ...   
bivalent_booster_65plus            1842112
bivalent_booster_65plus_pop_pct    1842112
combined_key                             0
population                               0
deaths                                   0
Length: 83, dtype: int64

In [ ]:
df.to_pickle(os.path.join(DATA_PATH, 'df.pkl'))